# Prep

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
exp_name = os.path.basename(os.path.dirname(os.getcwd()))
print(f"{exp_name=}")

# add code dir to path so I can import 
# if directory_path not in sys.path:
#     sys.path.insert(0, directory_path)

# enable auto reload files so I can catch instant changes in source file


import fiftyone as fo
print(f"{fo.__version__=}")
import wandb 
import fiftyone as fo
from fiftyone import ViewField as FF
import time
import torch
import json  
import glob
import analyze as A
import network as N
import utils as U

# import EpicKitchens50 as mydata

exp_name='exp_0019_202303031058'
fo.__version__='0.19.1'


/z/home/yayuanli/Research/darpa_ptg/darpa_ptg_yayuan/ptg_research/exp4/env_exp4/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# Prepare Fiftyone Dataset for experiment analysis

In [54]:
# copy from training
config = U.exp_config
config["device"] = "cpu"

print(f"Loading FO Dataset")
dataset_name = f"{config['dataset_prefix']}_FO_v{config['DATASET_FO_VERSION']}_{exp_name}"
if dataset_name in fo.list_datasets():
    print(f"exp fo dataset exist")
    dataset_raw = fo.load_dataset(dataset_name)
else:
    print(f"cloning exp dataset")
    dataset_raw = fo.load_dataset(f"{config['dataset_prefix']}_FO_v{config['DATASET_FO_VERSION']}").clone(dataset_name, persistent=True)
dataset_videos = dataset_raw.load_saved_view("videos")
dataset_blobs = dataset_raw.load_saved_view("blobs")
dataset_clips = dataset_raw.load_saved_view("clips")
dataset_clips.untag_samples("val")
dataset_clips.untag_samples("train")
ratio_val = 0.1
dataset_clips.take(int(ratio_val * len(dataset_clips)), seed=51).tag_samples("val")
dataset_clips.match_tags("val", bool=False).tag_samples("train")


Loading FO Dataset
exp fo dataset exist


In [55]:
dataset_raw.skeletons = {"pose": fo.KeypointSkeleton(labels=["WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "MEAN_ALL"], edges=[[0,1,2,3,4],[0,5,6,7,8],[9,10,11,12],[13,14,15,16],[0,17,18,19,20],[5,9,13,17]])}
dataset_raw.skeletons.update({"forecasting_groundtruth": fo.KeypointSkeleton(labels=["WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "MEAN_ALL"], edges=[[0,1,2,3,4],[0,5,6,7,8],[9,10,11,12],[13,14,15,16],[0,17,18,19,20],[5,9,13,17]])})

dataset_raw.save()

In [60]:
# dataset_clips_x = dataset_clips.clone()
# set tags
print(f"tagging")
for split in ["train", "val"]:
    split_clip_ids = []
    predictions = torch.load(f"{os.path.dirname(os.getcwd())}/outputs/best_predictions_{split}.json")
    sample_ids = list(predictions.keys())
    split_clip_ids = []
    for clip in dataset_clips:
        if clip.frames[clip.support[0]].id in sample_ids:
                split_clip_ids.append(clip.id)
        dataset_clips.select(split_clip_ids).tag_samples(f"{split}")

tagging


In [65]:
split = "train"
torch.load(f"{os.path.dirname(os.getcwd())}/outputs/best_predictions_{split}.json").keys()

dict_keys(['6401a6fd6beec2ccd0f6ca38', '6401a6fd6beec2ccd0f6c8e2', '6401a6fd6beec2ccd0f6c628', '6401a6fd6beec2ccd0f6c87c', '6401a6fd6beec2ccd0f6c7c6', '6401a6fd6beec2ccd0f6c84c', '6401a6fd6beec2ccd0f6c6f6', '6401a6fd6beec2ccd0f6c36f', '6401a6fd6beec2ccd0f6c44e', '6401a6fd6beec2ccd0f6c4e9'])

In [43]:
dataset_clips.values("id")

['64018ad2795a4bec62a12250',
 '64018ad2795a4bec62a12251',
 '64018ad2795a4bec62a12252',
 '64018ad2795a4bec62a1225d',
 '64018ad2795a4bec62a122bf',
 '64018ad2795a4bec62a1232e',
 '64018ad2795a4bec62a12357',
 '64018ad2795a4bec62a12362',
 '64018ad2795a4bec62a1236d',
 '64018ad3795a4bec62a12378',
 '64018ad3795a4bec62a12383']

In [45]:
dict(zip(*dataset_clips.values(["narration.label", "id"])))

{'open door': '64018ad2795a4bec62a12250',
 'turn on light': '64018ad2795a4bec62a12251',
 'open drawer': '64018ad2795a4bec62a12252',
 'close fridge': '64018ad3795a4bec62a12378',
 'close door': '64018ad2795a4bec62a122bf',
 'open fridge': '64018ad3795a4bec62a12383',
 'take celery': '64018ad2795a4bec62a12357',
 'take container': '64018ad2795a4bec62a12362',
 'take tofu': '64018ad2795a4bec62a1236d'}

# Load Predictions

In [53]:
if prediction_loaded == False:
    print(f"Analyzing")
    A.analyze(exp_name, config, dataset_raw, dataset_clips)
    print(f"DONE")
    prediction_loaded = True
else:
    print(f"Loaded Already")

Analyzing
split='train'


KeyError: '64018b048e2069c9f11d73cd'

# Forcast pose from crossed prompt
Get a sense, how open-voacb text guidance work

In [14]:
# net
net = N.get_network(config)
net.load_state_dict(torch.load(f"/z/home/yayuanli/Research/darpa_ptg/darpa_ptg_yayuan/ptg_research/exp2/outputs/experiments/{exp_name}/outputs/best_model_val.pt"))
net.eval();

In [15]:
# import pdb; pdb.set_trace()
prompts = [mc.task_spec[0][2], mc.task_spec[0][4]]
# one_step2_clip = dataset_clips.match_tags("val").filter_labels("frames.narration", FF("label")==mc.task_spec[0][2]).first().id
# one_step4_clip = dataset_clips.match_tags("val").filter_labels("frames.narration", FF("label")==mc.task_spec[0][4]).first().id
# dataset_clip_inf = dataset_clips.select([one_step4_clip, one_step2_clip])
dataset_clip_inf = dataset_clips.match_tags("val")
dataset_frame_inf = dataset_clip_inf.to_frames()

A.set_skeletons(dataset, prompts);
dataset.save()
A.set_skeletons(dataset_clip_inf, prompts);
dataset_clip_inf.save()

In [73]:
if crossed_prompt_inferenced == False:
    print(f"Inferencing")    
    frame_info = A.inference(net, config, dataset, dataset_clip_inf, dataset_frame_inf, prompts)
    print(f"Inference DONE")
    print("visualizing")
    A.visualize(dataset, dataset_clip_inf, config, frame_info)
    crossed_prompt_inferenced = True
else:
    print(f"inferenced already")

Inferencing
 711 [3.4m elapsed, 4.1 frame/s]  
Inference DONE
visualizing
 100% |███████████████████████| 8/8 [3.2m elapsed, 0s remaining, 0.0 clips/s]    


# Dive in: Check the cross attention maps

In [16]:
import matplotlib.pyplot as plt

# attention map
ts = 3
head = 1
self_cross = 0
attmap = list(frame_info["63e68f8dfe219f3d4c6b95ee"].values())[0]["attention_weights"][ts][head][self_cross]
plt.figure(dpi=500)
plt.imshow(attmap[:,0,:].permute(1,0).cpu().numpy(), cmap="Reds")
plt.show()

NameError: name 'frame_info' is not defined

# FO app

In [64]:
# session.view = dataset.select(dataset.values("id")[0]);
session.view = dataset.limit(1)

# forecasting process

In [62]:
# import pdb; pdb.set_trace()
forecasting_process_dir = "/".join(directory_path.split("/")[:-1]+["outputs", "forecasting_process"])
A.save_forecasting_process(dataset_frame_inf.limit(1), 
                           ["pose", "pose_pred_2UabktsnbftjSnbotl1iuate", "pose_pred_4UtktsjftjSjotnb"], 
                           forecasting_process_dir)

0
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8206a6ef04e2406cf4',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

1
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8306a6ef04e2406db2',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

2
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8406a6ef04e2406e70',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

3
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8406a6ef04e2406f2e',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

4
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8606a6ef04e2406fec',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

5
<Sample: {
    'id': None,
    'media_type': 'image',
    'filepath': '/z/dat/CookBook/MILLYCookbook_media_v000/A_pin/mevo/0/video-0006/pv_frames/frame_0000000234.jpg',
    'tags': ['val'],
    'metadata': None,
    'sample_id': '63e68e265e4a38f5520e6b7a',
    'frame_number': 235,
    'narration': <Classification: {
        'id': '64004b8706a6ef04e24070aa',
        'tags': [],
        'label': '2. Use a butter knife to scoop nut butter from the jar. Spread nut butter onto tortilla, leaving 1/2 inch uncovered at the edges.',
        'confidence': None,
        'logits': None,
    }>,
    'pose': <Keypoints: {
        'keypoints': [
            <Keypoint: {
                'id': '63b2115746a6069831767470',
                'attributes': {},
                'tags': [],
                'label': 'Left',
                'points': [
                    [0.2960015535354614, 0.8023756742477417],
                    [0.33296841382980347, 0.7667065858840942],
                    [0.3575604557991

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:




os.system("ffmpeg -framerate 30 -i /path/to/images/%d.jpg -c:v libx264 -profile:v high -pix_fmt yuv420p output.mp4")

In [87]:
os.makedirs(directory_path, exist_ok=True) 

In [88]:
directory_path

'/z/home/yayuanli/Research/darpa_ptg/darpa_ptg_yayuan/ptg_research/exp2/outputs/experiments/exp_0148_202302131348/code'

In [ ]:
os.path.dirname(directory_path)